In [ ]:
!pip install https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/en_legal_ner_trf-any-py3-none-any.whl
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl

In [ ]:
cd '/content/drive/MyDrive/NLP/NLP project/legal_NER-main'

/content/drive/MyDrive/NLP/NLP project/legal_NER-main


In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd

with open('./NER_TRAIN/NER_TRAIN_PREAMBLE.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

df.to_csv('csvfile.csv', encoding='utf-8', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df

,id,annotations,data,meta
0,d79fb7f965a74e418212458285c7c213,"[{'result': [{'value': {'start': 7, 'end': 40,...",{'text': 'In The High Court Of Kerala At Ernak...,{'source': 'criminal_kerala_high_court preambl...
1,10d82191b30e4e9d9b0ab91c34606631,"[{'result': [{'value': {'start': 21, 'end': 40...",{'text': 'In The Court Of Shri Lokesh Kumar Sh...,{'source': 'constitution_districtcourts preamb...
2,b150ace41ceb4acf87d12f75b4748ffc,"[{'result': [{'value': {'start': 11, 'end': 45...",{'text': 'Before The Madurai Bench Of Madras H...,{'source': 'civil_madras_high_court preamble h...
3,278f93b03a484705b3d96a72aa09be26,"[{'result': [{'value': {'start': 11, 'end': 45...",{'text': 'Before The Madurai Bench Of Madras H...,{'source': 'tax_madras_high_court preamble htt...
4,ceab6a1b200242208a4d6beb042d8743,"[{'result': [{'value': {'start': 85, 'end': 11...",{'text': '1 ...,{'source': 'criminal_bombay_high_court preambl...
...,...,...,...,...
1555,2b617380eac44e85a80d53e9c067360b,"[{'result': [{'value': {'start': 7, 'end': 41,...",{'text': 'In The High Court Of Judicature At M...,{'source': 'motorvehicles_madras_high_court pr...
1556,799133f87be04599a0b5367a4a1a23bf,"[{'result': [{'value': {'start': 12, 'end': 24...",{'text': 'Petitioner: Khudiram Das Vs. Resp...,{'source': 'criminal_supremecourts preamble ht...
1557,d21055fa778e4579a5711c6ffb1b9560,"[{'result': [{'value': {'start': 7, 'end': 40,...",{'text': 'In The High Court Of Judicature At P...,{'source': 'constitution_patna_high_court prea...
1558,307722b0639841c9b31418fbc84d7a52,"[{'result': [{'value': {'start': 16, 'end': 52...",{'text': 'Petitioner: The Automobile Transport...,{'source': 'tax_supremecourts preamble https:/...


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


## Text Preprocessing

### Preproceesing annotations start

In [ ]:
annotations_column = df['annotations']

In [ ]:
annotations_column =  annotations_column.to_numpy()

In [ ]:
annotation_start = []
annotation_end = []


for sent in annotations_column:
  sent_start = []
  sent_end = []
  sent_text = []
  sent_label = []
  for sent_res in sent:
    for res_value in sent_res['result']:
      sent_start.append(res_value['value']['start'])

    for res_value in sent_res['result']:
      sent_end.append(res_value['value']['end'])
    


  annotation_start.append(sent_start)
  annotation_end.append(sent_end)


In [ ]:
annotation_start[0]

[7, 73, 171, 237, 304, 350, 391, 425]

In [ ]:
len(annotation_end)

1560

In [ ]:
annotation_text = []
annotation_label = []
for sent in annotations_column:
    sent_text = []
    sent_label = []
    for sent_res in sent:

        for res_value in sent_res['result']:
          sent_text.append(res_value['value']['text'])
        
        for res_value in sent_res['result']:
          sent_label.append(res_value['value']['labels']) 

    annotation_text.append(sent_text)
    annotation_label.append(sent_label)

In [ ]:
annotation_text[0]

['High Court Of Kerala At Ernakulam',
 'T.R.Ajayan',
 'M.Ravindran',
 'Nirmala Dinesh',
 'A.Kumar',
 'M.K.Pushpalatha',
 'P.R.Raman',
 'V.K.Mohanan']

In [ ]:
annotation_label[0]

[['COURT'],
 ['PETITIONER'],
 ['RESPONDENT'],
 ['RESPONDENT'],
 ['LAWYER'],
 ['LAWYER'],
 ['JUDGE'],
 ['JUDGE']]

In [ ]:
column_names = ['start', 'end' , 'text' , 'labels']

final_df = pd.DataFrame()
final_df['start'] = annotation_start
final_df['end'] = annotation_end
final_df['text'] = annotation_text
final_df['labels'] = annotation_label

final_df.head()

,start,end,text,labels
0,"[7, 73, 171, 237, 304, 350, 391, 425]","[40, 83, 182, 251, 311, 365, 400, 436]","[High Court Of Kerala At Ernakulam, T.R.Ajayan...","[[COURT], [PETITIONER], [RESPONDENT], [RESPOND..."
1,"[21, 371, 445, 689, 741, 852, 1007, 1035, 1111...","[40, 383, 470, 698, 764, 869, 1031, 1047, 1121...","[Lokesh Kumar Sharma, Chanakyapuri, Jindal Ste...","[[JUDGE], [PETITIONER], [RESPONDENT], [RESPOND..."
2,"[11, 98, 192, 208, 219, 237, 253, 265, 281, 29...","[45, 115, 204, 215, 233, 249, 261, 277, 295, 3...","[Madurai Bench Of Madras High Court, V.Ramasub...","[[COURT], [JUDGE], [PETITIONER], [PETITIONER],..."
3,"[11, 99, 144, 305, 461, 593, 695, 1335, 1364, ...","[45, 112, 229, 330, 473, 604, 784, 1350, 1375,...","[Madurai Bench Of Madras High Court, T.Sudanth...","[[COURT], [JUDGE], [PETITIONER], [RESPONDENT],..."
4,"[85, 435, 614, 773, 879, 997, 1276, 1598, 1658...","[119, 460, 637, 798, 887, 1035, 1296, 1617, 16...","[High Court Of Judicature At Bombay, Motiram M...","[[COURT], [PETITIONER], [RESPONDENT], [PETITIO..."


In [ ]:
for a,b,c,d in zip(annotation_start[:2], annotation_end[:2],annotation_text[:2],annotation_label[:2]):
  print(a,b,c,d)

[7, 73, 171, 237, 304, 350, 391, 425] [40, 83, 182, 251, 311, 365, 400, 436] ['High Court Of Kerala At Ernakulam', 'T.R.Ajayan', 'M.Ravindran', 'Nirmala Dinesh', 'A.Kumar', 'M.K.Pushpalatha', 'P.R.Raman', 'V.K.Mohanan'] [['COURT'], ['PETITIONER'], ['RESPONDENT'], ['RESPONDENT'], ['LAWYER'], ['LAWYER'], ['JUDGE'], ['JUDGE']]
[21, 371, 445, 689, 741, 852, 1007, 1035, 1111, 1245] [40, 383, 470, 698, 764, 869, 1031, 1047, 1121, 1255] ['Lokesh Kumar Sharma', 'Chanakyapuri', 'Jindal Steel & Power Ltd.', 'The State', 'Planman Media Pvt. Ltd.', 'Arindam Chaudhuri', 'Jindal Steel & Power Ltd', 'State & Ors.', 'A. Sandeep', 'Ashok Bose'] [['JUDGE'], ['PETITIONER'], ['RESPONDENT'], ['RESPONDENT'], ['RESPONDENT'], ['RESPONDENT'], ['RESPONDENT'], ['RESPONDENT'], ['LAWYER'], ['LAWYER']]


In [ ]:
zip(annotation_start[0], annotation_end[0])

In [ ]:
pwd

'/content/drive/My Drive/NLP/NLP project/legal_NER-main'

In [ ]:
final_df.to_csv('Processed DataFrame')

## IOB tagging

In [ ]:
import pandas as pd

df_ = pd.read_csv('/content/drive/MyDrive/NLP/NLP project/legal_NER-main/Processed DataFrame')
df_.head()

,Unnamed: 0,start,end,text,labels
0,0,"[7, 73, 171, 237, 304, 350, 391, 425]","[40, 83, 182, 251, 311, 365, 400, 436]","['High Court Of Kerala At Ernakulam', 'T.R.Aja...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['..."
1,1,"[21, 371, 445, 689, 741, 852, 1007, 1035, 1111...","[40, 383, 470, 698, 764, 869, 1031, 1047, 1121...","['Lokesh Kumar Sharma', 'Chanakyapuri', 'Jinda...","[['JUDGE'], ['PETITIONER'], ['RESPONDENT'], ['..."
2,2,"[11, 98, 192, 208, 219, 237, 253, 265, 281, 29...","[45, 115, 204, 215, 233, 249, 261, 277, 295, 3...","['Madurai Bench Of Madras High Court', 'V.Rama...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['PETIT..."
3,3,"[11, 99, 144, 305, 461, 593, 695, 1335, 1364, ...","[45, 112, 229, 330, 473, 604, 784, 1350, 1375,...","['Madurai Bench Of Madras High Court', 'T.Suda...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['RESPO..."
4,4,"[85, 435, 614, 773, 879, 997, 1276, 1598, 1658...","[119, 460, 637, 798, 887, 1035, 1296, 1617, 16...","['High Court Of Judicature At Bombay', 'Motira...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['..."


In [ ]:
text = df.iloc[[0]]['data']

In [ ]:
text[0]['text']

"In The High Court Of Kerala At Ernakulam\n\nCrl Mc No. 1622 of 2006()\n\n\n1. T.R.Ajayan, S/O. O.Raman,\n                      ...  Petitioner\n\n                        Vs\n\n\n\n1. M.Ravindran,\n                       ...       Respondent\n\n2. Mrs. Nirmala Dinesh, W/O. Dinesh,\n\n                For Petitioner  :Sri.A.Kumar\n\n                For Respondent  :Smt.M.K.Pushpalatha\n\nThe Hon'ble Mr. Justice P.R.Raman\nThe Hon'ble Mr. Justice V.K.Mohanan\n\n Dated :07/01/2008\n\n O R D E R\n"

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(text[0]['text'])

for d in doc:
  print(d.text)

In
The
High
Court
Of
Kerala
At
Ernakulam



Crl
Mc
No
.
1622
of
2006
(
)




1
.
T.R.Ajayan
,
S
/
O.
O.Raman
,

                      
...
 
Petitioner


                        
Vs





1
.
M.Ravindran
,

                       
...
      
Respondent



2
.
Mrs.
Nirmala
Dinesh
,
W
/
O.
Dinesh
,


                
For
Petitioner
 
:
Sri
.
A.Kumar


                
For
Respondent
 
:
Smt
.
M.K.Pushpalatha



The
Hon'ble
Mr.
Justice
P.R.Raman


The
Hon'ble
Mr.
Justice
V.K.Mohanan


 
Dated
:
07/01/2008


 
O
R
D
E
R




In [ ]:
import re
t = re.sub('\n', " ", text[0]['text'])


In [ ]:
" ".join(t.split())

"In The High Court Of Kerala At Ernakulam Crl Mc No. 1622 of 2006() 1. T.R.Ajayan, S/O. O.Raman, ... Petitioner Vs 1. M.Ravindran, ... Respondent 2. Mrs. Nirmala Dinesh, W/O. Dinesh, For Petitioner :Sri.A.Kumar For Respondent :Smt.M.K.Pushpalatha The Hon'ble Mr. Justice P.R.Raman The Hon'ble Mr. Justice V.K.Mohanan Dated :07/01/2008 O R D E R"

In [ ]:
df['data']

0       {'text': 'In The High Court Of Kerala At Ernak...
1       {'text': 'In The Court Of Shri Lokesh Kumar Sh...
2       {'text': 'Before The Madurai Bench Of Madras H...
3       {'text': 'Before The Madurai Bench Of Madras H...
4       {'text': '1                                   ...
                              ...                        
1555    {'text': 'In The High Court Of Judicature At M...
1556    {'text': 'Petitioner:
Khudiram Das

	Vs.

Resp...
1557    {'text': 'In The High Court Of Judicature At P...
1558    {'text': 'Petitioner:
The Automobile Transport...
1559    {'text': 'Petitioner:
East India COMMERClAL Co...
Name: data, Length: 1560, dtype: object

In [ ]:
def preprocess(sent):
  sent = re.sub('\n', " ", sent)
  sent = sent.replace("...", " ")
  sent = " ".join(sent.split())

  return sent


clean_text = []
for i in df['data']:
  clean_text.append(preprocess(i['text']))

In [ ]:
len(clean_text)

1560

In [ ]:
df_['clean_text'] = clean_text

In [ ]:
df_.head()

,Unnamed: 0,start,end,text,labels,clean_text
0,0,"[7, 73, 171, 237, 304, 350, 391, 425]","[40, 83, 182, 251, 311, 365, 400, 436]","['High Court Of Kerala At Ernakulam', 'T.R.Aja...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['...",In The High Court Of Kerala At Ernakulam Crl M...
1,1,"[21, 371, 445, 689, 741, 852, 1007, 1035, 1111...","[40, 383, 470, 698, 764, 869, 1031, 1047, 1121...","['Lokesh Kumar Sharma', 'Chanakyapuri', 'Jinda...","[['JUDGE'], ['PETITIONER'], ['RESPONDENT'], ['...",In The Court Of Shri Lokesh Kumar Sharma Addit...
2,2,"[11, 98, 192, 208, 219, 237, 253, 265, 281, 29...","[45, 115, 204, 215, 233, 249, 261, 277, 295, 3...","['Madurai Bench Of Madras High Court', 'V.Rama...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['PETIT...",Before The Madurai Bench Of Madras High Court ...
3,3,"[11, 99, 144, 305, 461, 593, 695, 1335, 1364, ...","[45, 112, 229, 330, 473, 604, 784, 1350, 1375,...","['Madurai Bench Of Madras High Court', 'T.Suda...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['RESPO...",Before The Madurai Bench Of Madras High Court ...
4,4,"[85, 435, 614, 773, 879, 997, 1276, 1598, 1658...","[119, 460, 637, 798, 887, 1035, 1296, 1617, 16...","['High Court Of Judicature At Bombay', 'Motira...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['...","1 In The High Court Of Judicature At Bombay, N..."


In [ ]:
s = df_.iloc[0]['clean_text']

for i in range(7,40):
  print(s[i])

H
i
g
h
 
C
o
u
r
t
 
O
f
 
K
e
r
a
l
a
 
A
t
 
E
r
n
a
k
u
l
a
m


In [ ]:
df_.iloc[1]['clean_text']

import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(df_.iloc[0]['clean_text'])

print(doc[0])

In


In [ ]:
span = doc.char_span(73,100,label = "COURT")
span

## New solution

In [ ]:
def sentence_cleaner(sentence):
    '''
    replacing the unwanted charachters
    '''
    reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
    return " ".join(reg.findall(sentence.replace('\n', ' '))).replace(" / ", '/')

In [ ]:
def sentence_cleaner_v1(sentence):
    '''
    replacing the unwanted charachters
    '''
    sent_1 = " ".join(sent_1.replace("\n" , " ").replace("..." , " ").replace("-", "").split())
    doc = nlp(sent_1)
    doc_list = []
    for i in doc:
      doc_list.append(str(i))
    return doc_list

In [ ]:
def sentence_tagging(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = sentence_cleaner_v1(sent[ ann['value']['start']: ann['value']['end']])
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,temp_len)
    if temp_len > 1 :
        for i in range(temp_len):
            #print(i, temp_len - 1)
            if i == 0 :
              word2ent[temp_sent[i]] =  "B-" + ann['value']['labels'][0]
            
            elif i == (temp_len - 1):
              word2ent[temp_sent[i]] =  "E-" + ann['value']['labels'][0]

            else :
              word2ent[temp_sent[i]] =  "I-" + ann['value']['labels'][0]

    else :
        word2ent[temp_sent[0]] = "B-" + ann['value']['labels'][0]

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys() and word + ',' not in word2ent.keys() :
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
        tag_list.append(word2ent[word])
      
      else :
          tag_list.append(word2ent[word])
  
  
  print(word2ent )
  return tag_list
      

In [ ]:
def sentence_tagging_v1(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
      tag_list.append(word2ent[word])
      
  
  
  #print(word2ent )
  return tag_list
      

In [ ]:
IOBE_tagging = []
i = 0
for sent, anno in zip( df['data'].to_list(), df['annotations'].to_list()):
    print(i)
    i +=1 
    IOBE_tagging.append(sentence_tagging_v1(sent,anno))
    
  

In [ ]:
IOBE_tagging[0]

In [ ]:
for i in df['annotations'].to_list()[642][0]['result']:
  print(sent_1[i['value']['start'] : i['value']['end']] ,  i['value']['start']  , i['value']['end'])

Raj Kumar 21 30
Addl. District 
           Judge­09: Tis Hazari Courts : Delhi 32 94
Sudhir Kumar Sharma 177 196
Subhash Chander Sharma 206 228
Suman Kumar Sharma 239 257
Satish Chander Sharma 268 289
Sudesh Sharma 339 352
Sushila Sharma 407 421
Suresh C. Sharma 749 765
Sushil Pal Sharma 963 980
Land & Development Officer, 
       Nirman Bhawan, New Delhi 1090 1150


In [ ]:
sent_1 = df['data'].to_list()[0]['text']
sent_1

"In The High Court Of Kerala At Ernakulam\n\nCrl Mc No. 1622 of 2006()\n\n\n1. T.R.Ajayan, S/O. O.Raman,\n                      ...  Petitioner\n\n                        Vs\n\n\n\n1. M.Ravindran,\n                       ...       Respondent\n\n2. Mrs. Nirmala Dinesh, W/O. Dinesh,\n\n                For Petitioner  :Sri.A.Kumar\n\n                For Respondent  :Smt.M.K.Pushpalatha\n\nThe Hon'ble Mr. Justice P.R.Raman\nThe Hon'ble Mr. Justice V.K.Mohanan\n\n Dated :07/01/2008\n\n O R D E R\n"

In [ ]:
# [\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]

reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
" ".join(reg.findall(sent_1.replace('\n', ' '))).replace(" / ", '/')

In [ ]:
" ".join(sent_1.replace(" "," ").split())

In [ ]:
word2count = {}

for sent in df['data'].to_list():
    for word in sent['text'].split():
        if word not in word2count:
          word2count[word] = 0
        else:
          word2count[word] += 1

In [ ]:
for key in word2count.keys():
  if re.search(r'\bM+\b', key):
    print(key)

## Using Tokenizer

In [ ]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

In [ ]:
sent_1 = " ".join(sent_1.replace("\n" , " ").replace("..." , " ").replace("-", "").split())
doc = nlp(sent_1)
doc_list = []
for i in doc:
  doc_list.append(str(i))
doc_list

['In',
 'The',
 'High',
 'Court',
 'Of',
 'Kerala',
 'At',
 'Ernakulam',
 'Crl',
 'Mc',
 'No',
 '.',
 '1622',
 'of',
 '2006',
 '(',
 ')',
 '1',
 '.',
 'T.R.Ajayan',
 ',',
 'S',
 '/',
 'O.',
 'O.Raman',
 ',',
 'Petitioner',
 'Vs',
 '1',
 '.',
 'M.Ravindran',
 ',',
 'Respondent',
 '2',
 '.',
 'Mrs.',
 'Nirmala',
 'Dinesh',
 ',',
 'W',
 '/',
 'O.',
 'Dinesh',
 ',',
 'For',
 'Petitioner',
 ':',
 'Sri',
 '.',
 'A.Kumar',
 'For',
 'Respondent',
 ':',
 'Smt',
 '.',
 'M.K.Pushpalatha',
 'The',
 "Hon'ble",
 'Mr.',
 'Justice',
 'P.R.Raman',
 'The',
 "Hon'ble",
 'Mr.',
 'Justice',
 'V.K.Mohanan',
 'Dated',
 ':',
 '07/01/2008',
 'O',
 'R',
 'D',
 'E',
 'R']